In [ ]:
import ibis
from ibis import _
import pydeck as pdk
from boilerplate import *

con = ibis.duckdb.connect(extensions=['httpfs', 'spatial', 'h3'])

In [ ]:
set_secrets(con)
gbif = con.read_parquet("s3://cboettig/gbif/2024-10-01/**")

In [ ]:
%%time

dest = "csv/gbif_demo.csv"

(gbif
  .filter(_["class"].isin(["Insecta"]))
 # .filter(_.species == "Balaenoptera musculus")
  .rename(hex = "h5")
  .group_by(_.hex)
  .agg(n = _.count())
  .mutate(logn = _.n.log())
  .mutate(value = (255 * _.logn / _.logn.max()).cast("int"))
  .to_csv("s3://cboettig/gbif/" + dest)
)

In [ ]:
url = "https://minio.carlboettiger.info/cboettig/gbif/" + dest

layer = HexagonLayer(url)
deck = DeckGlobe(layer)

deck.to_html("globe.html")


---


# Additional examples

In [ ]:
## Filter to specific polygon
import geopandas as gpd
polygons = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/blob/master/geojson/ne_110m_admin_0_countries.geojson?raw=true")
polygon = polygons[polygons.NAME == 'Portugal']
poly_expr = ibis.literal(polygon.geometry.iloc[0])


In [ ]:
name = "portugal_bird_richness"
s3_csv = "s3://cboettig/gbif/csv/" + name + ".csv"
s3out = "s3://cboettig/gbif/" + name + ".pmtiles"

subset = (gbif
  .filter( _.geom.within(poly_expr))
  .filter(_["class"] == "Aves")
  .rename(hex = "h8")
  .select(_.hex, _.species).distinct() # distinct species
  .group_by(_.hex)
  .agg(n = _.count())
  .mutate(wkt =  h3_cell_to_boundary_wkt(_.hex)) # write to pmtiles?
  .mutate(logn = _.n.log())
  .mutate(value = (255 * _.logn / _.logn.max()).cast("int"))
  .to_geo(s3out)
)
